# Football Object Detection with Tactical Map Position Estimation

In [1]:
# Import libraries
import numpy as np
import pandas as pd

import cv2
import skimage
from PIL import Image
from ultralytics import YOLO
from sklearn.metrics import mean_squared_error
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
import torch

import json
import yaml
import time

In [2]:
# Get tactical map keypoints positions dictionary
json_path = "configs/pitch map labels position.json"
with open(json_path, 'r') as f:
    keypoints_map_pos = json.load(f)

# Get football field keypoints numerical to alphabetical mapping
yaml_path = "configs/config pitch dataset.yaml"
with open(yaml_path, 'r') as file:
    classes_names_dic = yaml.safe_load(file)
classes_names_dic = classes_names_dic['names']

# Get football field keypoints numerical to alphabetical mapping
yaml_path = "configs/config players dataset.yaml"
with open(yaml_path, 'r') as file:
    labels_dic = yaml.safe_load(file)
labels_dic = labels_dic['names']

print("Known coordinates of each keypoint on the tactical map:")
display(pd.DataFrame(keypoints_map_pos, index=['x','y']))
print("Numerical label of field keypoints (as defined when training the Yolo model):")
display(pd.Series(classes_names_dic, name='alpha_label').reset_index().rename({"index":"num_label"}, axis=1).set_index("alpha_label").transpose())
print("Numerical label of the player, referee, and ball objects (as defined when training the Yolo model):")
display(pd.Series(labels_dic, name='alpha_label').reset_index().rename({"index":"num_label"}, axis=1).set_index("alpha_label").transpose())
print('\033[1mThe dataframe representation are not used in what follows (original dictionary will be used)')

Known coordinates of each keypoint on the tactical map:


,TLC,TRC,TR6MC,TL6MC,TR6ML,TL6ML,TR18MC,TL18MC,TR18ML,TL18ML,...,BR6MC,BL6MC,BR6ML,BL6ML,BR18MC,BL18MC,BR18ML,BL18ML,BRArc,BLArc
x,15,291,188,116,189,116,221,84,221,84,...,189,117,189,117,221,84,221,84,182,121
y,15,15,35,35,15,15,73,73,15,15,...,501,501,521,521,463,463,521,521,463,463


Numerical label of field keypoints (as defined when training the Yolo model):


alpha_label,TLC,TRC,TR6MC,TL6MC,TR6ML,TL6ML,TR18MC,TL18MC,TR18ML,TL18ML,...,BR6MC,BL6MC,BR6ML,BL6ML,BR18MC,BL18MC,BR18ML,BL18ML,BRArc,BLArc
num_label,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27


Numerical label of the player, referee, and ball objects (as defined when training the Yolo model):


alpha_label,player,referee,ball
num_label,0,1,2


The dataframe representation are not used in what follows (original dictionary will be used)


In [3]:
# Set video path
video_path = 'test vid.mp4'

# Read tactical map image
tac_map = cv2.imread('tactical map.jpg')

# Define team colors (based on chosen video)
nbr_team_colors = 2
colors_dic = {
    "Chelsea":[(41,71,138), (220,98,88)], # Chelsea colors (Players kit color, GK kit color)
    "Man City":[(144,200,255), (188,199,3)] # Man City colors (Players kit color, GK kit color)
}

colors_list = colors_dic["Chelsea"]+colors_dic["Man City"] # Define color list to be used for detected player team prediction
color_list_lab = [skimage.color.rgb2lab([i/255 for i in c]) for c in colors_list] # Converting color_list to L*a*b* space

In [4]:
# Load the YOLOv8 players detection model
model_players = YOLO("models/players.pt")

# Load the YOLOv8 field keypoints detection model
model_keypoints = YOLO("models/field_keypoints.pt")

In [5]:
def filter_detections_by_class(detections, class_name):
    if class_name == 'player':
        class_number = 0
    elif class_name == 'referee':
        class_number = 1
    elif class_name == 'ball':
        class_number = 2
    list = []
    for detection in detections:
        if detection.boxes.cls == class_number:
            list.append(detection)
    return list


def get_player_in_possession(player_detections, ball_detections):
    if len(ball_detections) != 1:
        print(ball_detections)
        return None,None
    ball_x = ball_detections[0].boxes.xywh[0][0]
    ball_y = ball_detections[0].boxes.xywh[0][1]
    min_distance = 100
    idx = -100
    i = 0
    for player_detection in player_detections:
        player_x = player_detection.boxes.xywh[0][0]
        player_y = player_detection.boxes.xywh[0][1]
        distance = (player_x-ball_x) + abs(player_y-ball_y)
        if distance < min_distance :
            min_distance = distance
            idx = i
        i += 1
    print(player_detections[idx].tracker_id,idx)
    return player_detections[idx].tracker_id,idx

def detections2boxes(detections, confidence = True):
    list_of_boxes = []
    for detection in detections:
        list = detection.boxes.xyxy.tolist()
        if confidence:
            list[0].append(detection.boxes.conf.item())
        # list to tensor
        list_of_boxes.append(list)
    return np.squeeze(np.array(list_of_boxes ,dtype = float))


def tracks2boxes(tracks):
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


def match_detections_with_tracks(detections, tracks):
    # list of detections
    # np.array of boxes
    detection_boxes = detections2boxes(detections=detections, confidence=False)
    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)
    
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            detections[detection_index].tracker_id = tracks[tracker_index].track_id
        else :
            del detections[detection_index]
    for idx,obj in enumerate(detections):
        if not hasattr(obj,'tracker_id') :
           del detections[idx] 
    return detections

In [6]:
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False


In [7]:
# e7na ma3ana el id beta3 kol object wel ball holder
# ama te7aded el team color assign it
# the you say the ball now is with player with id x and in the y team 
# when the prev ball holder changes 
# if the ball 

In [8]:
# around the first box in the tracks plot id 1 and the second plot id 2 and so on 
# awel frame assign ids for each player or ref detected
# then ball posession
# then detect passes
# when ball possesor is changed:
# if different teams then steal
# if same team then pass
# array of colors each item corresponds to the id of a team
# el tabdeel

In [9]:
# 3awzeen neddy each player detected an id 
# we variable the ball holder to be an id of a player of these players

In [10]:
# in the first we assign an id to each player detected
# matching based on position and being being similar

In [11]:
from deep_sort_realtime.deepsort_tracker import DeepSort

object_tracker = DeepSort(max_age=5,
                n_init=2,
                nms_max_overlap=1.0,
                max_cosine_distance=0.3,
                nn_budget=None,
                override_track_class=None,
                embedder="mobilenet",
                half=True,
                bgr=True,
                embedder_gpu=True,
                embedder_model_name=None,
                embedder_wts=None,
                polygon=False,
                today=None)


In [13]:
# remove the color part and associate the id with the color ,,, donee
# we need to link player names with ids : dict with dict[id] = name ,,, hard code it
# we need to put the scores in the app when we complete 

In [12]:
def predict_team(player,frame_rgb):
    bbox = player.boxes.xyxy.cpu().numpy()[0]
    obj_img = frame_rgb[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
    obj_img_w, obj_img_h = obj_img.shape[1], obj_img.shape[0]
    center_filter_x1 = np.max([(obj_img_w//3), 1])
    center_filter_x2 = (2*obj_img_w//3)
    center_filter_y1 = np.max([(obj_img_h//6), 1])
    center_filter_y2 = (obj_img_h//3)
    center_filter = obj_img[center_filter_y1:center_filter_y2, 
                            center_filter_x1:center_filter_x2]

    rgb_avg = np.sum(np.sum(center_filter,axis=1),axis=0)/(center_filter.shape[0]*center_filter.shape[1]) 
    lab_avg = skimage.color.rgb2lab(rgb_avg/255)
    team_idx = np.argmin(np.sum(np.abs(color_list_lab - lab_avg),axis=1))
    return team_idx

In [ ]:
# Open video file
cap = cv2.VideoCapture(video_path)

#Initialize tracker
byte_tracker = BYTETracker(BYTETrackerArgs())

# Initialize frame counter
frame_nbr = 0

# Set keypoints average displacement tolerance level (in pixels) [set to -1 to always update homography matrix]
keypoints_displacement_mean_tol = 10

# Set confidence thresholds for players and field keypoints detections
player_model_conf_thresh = 0.60
keypoints_model_conf_thresh = 0.70

# Set variable to record the time when we processed last frame 
prev_frame_time = 0
# Set variable to record the time at which we processed current frame 
new_frame_time = 0

# Store the ball track history
ball_track_history = {'src':[],
                      'dst':[]
}

# Count consecutive frames with no ball detected
nbr_frames_no_ball = 0
# Threshold for number of frames with no ball to reset ball track (frames)
nbr_frames_no_ball_thresh = 30
# Distance threshold for ball tracking (pixels)
ball_track_dist_thresh = 100
# Maximum ball track length (detections)
max_track_length = 35


prev_team = -1
prev_id = -1
prev_idx = -1
team_with_ball = -1
player_with_ball_id = -1
player_with_ball_idx = -1
last_frame = -1


# Capture the start time
start = time.time()

# text
text = ""
#team dict
team = {}
# Loop through the video frames
while cap.isOpened():

    # Update frame counter
    frame_nbr += 1

    # Read a frame from the video
    success, frame = cap.read()

    # Copy image
    tac_map_copy = tac_map.copy()

    # Reset ball tracks if ball wasnt detected for many frames
    if nbr_frames_no_ball>nbr_frames_no_ball_thresh:
            ball_track_history['dst'] = []
            ball_track_history['src'] = []

    if success:
        
        #################### Part 1 ####################
        # Object Detection & Coordiante Transofrmation #
        ################################################

        # players, ref, balls bounding box
        results_players = model_players(frame, conf=player_model_conf_thresh)
        # keypoints bounding boxes
        results_keypoints = model_keypoints(frame, conf=keypoints_model_conf_thresh)

        # filter classes
        player_detections = filter_detections_by_class(detections=results_players[0], class_name="player")
        referee_detections = filter_detections_by_class(detections=results_players[0], class_name="referee")
        ball_detections = filter_detections_by_class(detections=results_players[0], class_name="ball")
        #tracked_detections = player_detections + referee_detections

        # update tracks
        tracks = byte_tracker.update(
        output_results=detections2boxes(detections=player_detections),
        img_info=frame.shape,
        img_size=frame.shape
        )

        player_detections = match_detections_with_tracks(detections=player_detections, tracks=tracks)

        ## Extract detections information
        bboxes_p = results_players[0].boxes.xyxy.cpu().numpy()                          # Detected players, referees and ball (x,y,x,y) bounding boxes
        bboxes_p_c = results_players[0].boxes.xywh.cpu().numpy()                        # Detected players, referees and ball (x,y,w,h) bounding boxes    
        labels_p = list(results_players[0].boxes.cls.cpu().numpy())                     # Detected players, referees and ball labels list
        confs_p = list(results_players[0].boxes.conf.cpu().numpy())                     # Detected players, referees and ball confidence level
        
        bboxes_k = results_keypoints[0].boxes.xyxy.cpu().numpy()                        # Detected field keypoints (x,y,w,h) bounding boxes
        bboxes_k_c = results_keypoints[0].boxes.xywh.cpu().numpy()                      # Detected field keypoints (x,y,w,h) bounding boxes
        labels_k = list(results_keypoints[0].boxes.cls.cpu().numpy())                   # Detected field keypoints labels list

        # from 0,1.. -> TLC,TRC..
        detected_labels = [classes_names_dic[i] for i in labels_k]

        # coordinates of keypoints in field
        detected_labels_src_pts = np.array([list(np.round(bboxes_k_c[i][:2]).astype(int)) for i in range(bboxes_k_c.shape[0])])

        # coordinates of mapped keypoints
        detected_labels_dst_pts = np.array([keypoints_map_pos[i] for i in detected_labels])


        ## if points detected > 3 map points and players to field map
        if len(detected_labels) > 3: 
            if frame_nbr > 1:
                # Determine common detected field keypoints between previous and current frames
                common_labels = set(detected_labels_prev) & set(detected_labels)
                # When at least 4 common keypoints are detected, determine if they are displaced on average beyond a certain tolerance level
                if len(common_labels) > 3:
                    common_label_idx_prev = [detected_labels_prev.index(i) for i in common_labels]   # Get labels indexes of common detected keypoints from previous frame
                    common_label_idx_curr = [detected_labels.index(i) for i in common_labels]        # Get labels indexes of common detected keypoints from current frame
                    coor_common_label_prev = detected_labels_src_pts_prev[common_label_idx_prev]     # Get labels coordiantes of common detected keypoints from previous frame
                    coor_common_label_curr = detected_labels_src_pts[common_label_idx_curr]          # Get labels coordiantes of common detected keypoints from current frame
                    coor_error = mean_squared_error(coor_common_label_prev, coor_common_label_curr)  # Calculate error between previous and current common keypoints coordinates
                    update_homography = coor_error > keypoints_displacement_mean_tol                 # Check if error surpassed the predefined tolerance level
                else:
                    update_homography = True                                                         
            else:
                update_homography = True

            if  update_homography:
                h, mask = cv2.findHomography(detected_labels_src_pts,                   # Calculate homography matrix
                                              detected_labels_dst_pts)                  
            
            detected_labels_prev = detected_labels.copy()                               # Save current detected keypoint labels for next frame
            detected_labels_src_pts_prev = detected_labels_src_pts.copy()               # Save current detected keypoint to measure error in next frame

            # get boxes of players and ball to map to the field map
            bboxes_p_c_0 = bboxes_p_c[[i==0 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected players (label 0)
            bboxes_p_c_2 = bboxes_p_c[[i==2 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected ball(s) (label 2)

            
            # Get coordinates of detected players on frame (x_center, y_center+h/2)
            detected_ppos_src_pts = bboxes_p_c_0[:,:2]  + np.array([[0]*bboxes_p_c_0.shape[0], bboxes_p_c_0[:,3]/2]).transpose()
            # Get coordinates of the first detected ball (x_center, y_center)
            detected_ball_src_pos = bboxes_p_c_2[0,:2] if bboxes_p_c_2.shape[0]>0 else None

            # Transform players coordinates from frame plane to tactical map plane using the calculated Homography matrix
            pred_dst_pts = []                                                           # Initialize players tactical map coordiantes list
            for pt in detected_ppos_src_pts: # Loop over players frame coordiantes
                # track players how mosh areff
                pt = np.append(np.array(pt), np.array([1]), axis=0)                     # Covert to homogeneous coordiantes
                dest_point = np.matmul(h, np.transpose(pt))                              # Apply homography transofrmation
                dest_point = dest_point/dest_point[2]                                   # Revert to 2D-coordiantes
                pred_dst_pts.append(list(np.transpose(dest_point)[:2]))                 # Update players tactical map coordiantes list
            pred_dst_pts = np.array(pred_dst_pts)

            # Transform ball coordinates from frame plane to tactical map plane using the calculated Homography matrix
            if detected_ball_src_pos is not None:
                pt = np.append(np.array(detected_ball_src_pos), np.array([1]), axis=0)
                dest_point = np.matmul(h, np.transpose(pt))
                dest_point = dest_point/dest_point[2]
                detected_ball_dst_pos = np.transpose(dest_point)

                # Update track ball position history to draw the ball 
                if len(ball_track_history['src'])>0 :
                    if np.linalg.norm(detected_ball_src_pos-ball_track_history['src'][-1])<ball_track_dist_thresh:
                        ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                        ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
                    else:
                        ball_track_history['src']=[(int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1]))]
                        ball_track_history['dst']=[(int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1]))]
                else:
                    ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                    ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
            # Remove oldest tracked ball postion if track exceedes threshold        
            if len(ball_track_history) > max_track_length:
                    ball_track_history['src'].pop(0)
                    ball_track_history['dst'].pop(0)
            # track ball is closest to which player and assign it to a player when it changes players trigger  

        ######### Part 2 ########## 
        # Players Team Prediction #
        ########################### 
        if ((frame_nbr-1) % 10 == 0):
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)                                      # Convert frame to RGB
            obj_palette_list = []                                                                   # Initialize players color palette list
            palette_interval = (0,5)                                                                # Color interval to extract from dominant colors palette (1rd to 5th color)
            annotated_frame = frame                                                                 # Create annotated frame 
    
            for player in player_detections:
                team_idx = predict_team(player,frame_rgb)
                team[player.tracker_id] = team_idx
                player.team = list(colors_dic.keys())[team_idx//2] # "chelsea" or "man city"
                if team_idx % 2 == 0:
                    player.position = "Player"
                else :
                    player.position = "Goal Keeper"
        else :
            annotated_frame = frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            players_teams_list = []
            for player in player_detections:
                try :    
                    team_idx = team[player.tracker_id]
                except:
                    team_idx = predict_team(player,frame_rgb)
                player.team = list(colors_dic.keys())[team_idx//2] # "chelsea" or "man city"
                if team_idx % 2 == 0:
                    player.position = "Player"
                else :
                    player.position = "Goal Keeper"
                players_teams_list.append(team_idx)
                
            
        #################### Part 3 #####################
        # Updated Frame & Tactical Map With Annotations #
        #################################################
        for player in player_detections:
            bbox = player.boxes.xyxy.numpy()[0]
            annotated_frame = cv2.rectangle(annotated_frame, (int(bbox[0]), int(bbox[1])),  
                                        (int(bbox[2]), int(bbox[3])), (0,0,0), 1)
            if player.team == list(colors_dic.keys())[0]:
                color = (0,0,0)
            else :
                color = (255,255,255)
            cv2.putText(annotated_frame,f" {player.tracker_id}",(int(bbox[0]), int(bbox[1])-10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        ball_color_bgr = (0,0,255)                                                                          # Color (GBR) for ball annotation on tactical map
        j=0                                                                                                 # Initializing counter of detected players
        palette_box_size = 10                                                                               # Set color box size in pixels (for display)
        

        # Loop over all detected object by players detection model
        for i in range(bboxes_p.shape[0]):
            conf = confs_p[i]                                                                               # Get confidence of current detected object
            if labels_p[i]==0:                                                                              # Display annotation for detected players (label 0)

                
                color_rgb = colors_dic["Chelsea"][0]                                                        # Get detected player team color
                color_bgr = color_rgb[::-1]
                # Add tactical map player postion color coded annotation if more than 3 field keypoints are detected
                if len(detected_labels_src_pts)>3:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(pred_dst_pts[j][0]),int(pred_dst_pts[j][1])),
                                          radius=5, color=color_bgr, thickness=-1)

                j+=1                        # Update players counter
            else:                                                                                           # Display annotation for otehr detections (label 1, 2)
                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,0]), int(bboxes_p[i,1])), 
                                               (int(bboxes_p[i,2]), int(bboxes_p[i,3])), (255,255,255), 1)
                cv2.putText(annotated_frame, labels_dic[labels_p[i]] + f" {conf:.2f}",(int(bboxes_p[i,0]), int(bboxes_p[i,1])-10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

                # Add tactical map ball postion annotation if detected
                if detected_ball_src_pos is not None:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(detected_ball_dst_pos[0]), 
                                                   int(detected_ball_dst_pos[1])), radius=5, 
                                                   color=ball_color_bgr, thickness=3)
        for i in range(bboxes_k.shape[0]):
            annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_k[i,0]), int(bboxes_k[i,1])),  
                                        (int(bboxes_k[i,2]), int(bboxes_k[i,3])), (0,0,0), 1)
        
        # Plot the ball tracks on tactical map
        if len(ball_track_history['src'])>0:
            points = np.hstack(ball_track_history['dst']).astype(np.int32).reshape((-1, 1, 2))
            tac_map_copy = cv2.polylines(tac_map_copy, [points], isClosed=False, color=(0, 0, 100), thickness=2)
        
        # prev_team = undefined
        # prev_id = undefined
        # prev_idx = undefined
        # team_with_ball = undefined
        # player_with_ball_id = changed
        # player_with_ball_idx = changed 


        if len(ball_detections) == 1 :
            player_with_ball_id,player_with_ball_idx = get_player_in_possession(player_detections,ball_detections)
            team_with_ball = player_detections[player_with_ball_idx].team

        if abs(frame_nbr - last_frame) > 20:
            last_frame = frame_nbr
            current_position = player_detections[player_with_ball_idx].position
            
            # Player possession change detected
            if prev_id != player_with_ball_id:
                if prev_id == -1:
                    text += f"the ball is now with player {player_with_ball_id} of {team_with_ball}"
                
                else:
                    if prev_team != team_with_ball:
                        text += f"the ball was intercepted from {prev_team} to player {player_with_ball_id} of {team_with_ball}"
                    else:
                        if prev_position == "Goal Keeper":
                            text += f"the ball was passed from the Goal Keeper of {team_with_ball} to player {player_with_ball_id}"
                        elif current_position == "Goal Keeper":
                            text += f"the ball was passed from player {prev_id} to the Goal Keeper of {team_with_ball}"
                        else:
                            text += f"the ball was passed from player {prev_id} to player {player_with_ball_id} of {team_with_ball}"
            
            else:
                text += f"the ball is still with player number {player_with_ball_id}"
                
            # Update previous state
            prev_id, prev_team, prev_position = player_with_ball_id, team_with_ball, current_position
            print(prev_id,player_with_ball_id)




        
        # Combine annotated frame and tactical map in one image with colored border separation
        border_color = [255,255,255]                                                                        # Set border color (BGR)
        annotated_frame=cv2.copyMakeBorder(annotated_frame, 40, 10, 10, 10,                                 # Add borders to annotated frame
                                            cv2.BORDER_CONSTANT, value=border_color)
        tac_map_copy = cv2.copyMakeBorder(tac_map_copy, 70, 50, 10, 10, cv2.BORDER_CONSTANT,                # Add borders to tactical map 
                                           value=border_color)      
        tac_map_copy = cv2.resize(tac_map_copy, (tac_map_copy.shape[1], annotated_frame.shape[0]))          # Resize tactical map
        final_img = cv2.hconcat((annotated_frame, tac_map_copy))                                            # Concatenate both images
        ## Add info annotation
        cv2.putText(final_img, "Tactical Map", (1370,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        cv2.putText(final_img, "Press 'p' to pause & 'q' to quit", (820,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)

        new_frame_time = time.time()                                                                        # Get time after finished processing current frame
        fps = 1/(new_frame_time-prev_frame_time)                                                            # Calculate FPS as 1/(frame proceesing duration)
        prev_frame_time = new_frame_time                                                                    # Save current time to be used in next frame
        cv2.putText(final_img, "FPS: " + str(int(fps)), (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        
        # Display the final annotated frame
        cv2.imshow("YOLOv8 Players and Field Keypoints Detection with Team Prediction and Tactical Map",    
                    final_img)

        # Treat keyboard user inputs ("p" for pause/unpause & "q" for quit)
        key = cv2.waitKey(1)
        # Break the loop if 'q' is pressed
        if key == ord("q"):
            break
        if key == ord('p'):
            cv2.waitKey(-1) #wait until any key is pressed
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 19 players, 1 referee, 1 ball, 273.6ms
Speed: 2.6ms preprocess, 273.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 BL18MC, 143.5ms
Speed: 1.2ms preprocess, 143.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


(19, 5)
19 18



0: 384x640 19 players, 1 referee, 268.9ms
Speed: 1.3ms preprocess, 268.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 BL18MC, 149.0ms
Speed: 1.3ms preprocess, 149.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



(19, 5)


0: 384x640 18 players, 1 referee, 1 ball, 240.7ms
Speed: 1.2ms preprocess, 240.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 BL18MC, 132.5ms
Speed: 1.1ms preprocess, 132.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



...

0: 384x640 17 players, 1 referee, 1 ball, 253.5ms
Speed: 1.1ms preprocess, 253.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 LMC, 1 BL18MC, 1 BLArc, 155.9ms
Speed: 1.5ms preprocess, 155.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



(17, 5)
19 1


0: 384x640 17 players, 1 referee, 1 ball, 244.4ms
Speed: 1.2ms preprocess, 244.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 LMC, 1 BL18MC, 1 BLArc, 137.8ms
Speed: 1.0ms preprocess, 137.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



(17, 5)
19 1


0: 384x640 18 players, 1 referee, 1 ball, 240.7ms
Speed: 1.1ms preprocess, 240.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TLC, 1 TL6MC, 1 TL6ML, 1 TL18MC, 1 TL18ML, 1 TLArc, 1 LMC, 1 LML, 1 BL18MC, 1 BLArc, 137.3ms
Speed: 1.0ms preprocess, 137.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)



In [12]:
text
# awel ma id beye5tefy save object fe list we ba3deein awel ma tela2y id tany gayy shofo a2rab le anhy wa7ed min el da3o we assign el id da leeh
# keep track of the score and make it change manually

'the ball is now with player 3 of Chelseathe ball was passed from player 3 to player 19 of Chelseathe ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19the ball is still with player number 19'

In [1]:
# Open video file
cap = cv2.VideoCapture(video_path)

#Initialize tracker
byte_tracker = BYTETracker(BYTETrackerArgs())

# Initialize frame counter
frame_nbr = 0

# Set keypoints average displacement tolerance level (in pixels) [set to -1 to always update homography matrix]
keypoints_displacement_mean_tol = 10

# Set confidence thresholds for players and field keypoints detections
player_model_conf_thresh = 0.60
keypoints_model_conf_thresh = 0.70

# Set variable to record the time when we processed last frame 
prev_frame_time = 0
# Set variable to record the time at which we processed current frame 
new_frame_time = 0

# Store the ball track history
ball_track_history = {'src':[],
                      'dst':[]
}

# Count consecutive frames with no ball detected
nbr_frames_no_ball = 0
# Threshold for number of frames with no ball to reset ball track (frames)
nbr_frames_no_ball_thresh = 30
# Distance threshold for ball tracking (pixels)
ball_track_dist_thresh = 100
# Maximum ball track length (detections)
max_track_length = 35


prev_team = -1
prev_id = -1
prev_idx = -1
team_with_ball = -1
player_with_ball_id = -1
player_with_ball_idx = -1
last_frame = -1


# Capture the start time
start = time.time()

# text
text = ""
#team dict
team = {}
# Loop through the video frames
while cap.isOpened():

    # Update frame counter
    frame_nbr += 1

    # Read a frame from the video
    success, frame = cap.read()

    # Copy image
    tac_map_copy = tac_map.copy()

    # Reset ball tracks if ball wasnt detected for many frames
    if nbr_frames_no_ball>nbr_frames_no_ball_thresh:
            ball_track_history['dst'] = []
            ball_track_history['src'] = []

    if success:
        
        #################### Part 1 ####################
        # Object Detection & Coordiante Transofrmation #
        ################################################

        # players, ref, balls bounding box
        results_players = model_players(frame, conf=player_model_conf_thresh)
        # keypoints bounding boxes
        results_keypoints = model_keypoints(frame, conf=keypoints_model_conf_thresh)

        # filter classes
        player_detections = filter_detections_by_class(detections=results_players[0], class_name="player")
        referee_detections = filter_detections_by_class(detections=results_players[0], class_name="referee")
        ball_detections = filter_detections_by_class(detections=results_players[0], class_name="ball")
        #tracked_detections = player_detections + referee_detections

        # update tracks
        tracks = byte_tracker.update(
        output_results=detections2boxes(detections=player_detections),
        img_info=frame.shape,
        img_size=frame.shape
        )

        player_detections = match_detections_with_tracks(detections=player_detections, tracks=tracks)

        ## Extract detections information
        bboxes_p = results_players[0].boxes.xyxy.cpu().numpy()                          # Detected players, referees and ball (x,y,x,y) bounding boxes
        bboxes_p_c = results_players[0].boxes.xywh.cpu().numpy()                        # Detected players, referees and ball (x,y,w,h) bounding boxes    
        labels_p = list(results_players[0].boxes.cls.cpu().numpy())                     # Detected players, referees and ball labels list
        confs_p = list(results_players[0].boxes.conf.cpu().numpy())                     # Detected players, referees and ball confidence level
        
        bboxes_k = results_keypoints[0].boxes.xyxy.cpu().numpy()                        # Detected field keypoints (x,y,w,h) bounding boxes
        bboxes_k_c = results_keypoints[0].boxes.xywh.cpu().numpy()                      # Detected field keypoints (x,y,w,h) bounding boxes
        labels_k = list(results_keypoints[0].boxes.cls.cpu().numpy())                   # Detected field keypoints labels list

        # from 0,1.. -> TLC,TRC..
        detected_labels = [classes_names_dic[i] for i in labels_k]

        # coordinates of keypoints in field
        detected_labels_src_pts = np.array([list(np.round(bboxes_k_c[i][:2]).astype(int)) for i in range(bboxes_k_c.shape[0])])

        # coordinates of mapped keypoints
        detected_labels_dst_pts = np.array([keypoints_map_pos[i] for i in detected_labels])


        ## if points detected > 3 map points and players to field map
        if len(detected_labels) > 3: 
            if frame_nbr > 1:
                # Determine common detected field keypoints between previous and current frames
                common_labels = set(detected_labels_prev) & set(detected_labels)
                # When at least 4 common keypoints are detected, determine if they are displaced on average beyond a certain tolerance level
                if len(common_labels) > 3:
                    common_label_idx_prev = [detected_labels_prev.index(i) for i in common_labels]   # Get labels indexes of common detected keypoints from previous frame
                    common_label_idx_curr = [detected_labels.index(i) for i in common_labels]        # Get labels indexes of common detected keypoints from current frame
                    coor_common_label_prev = detected_labels_src_pts_prev[common_label_idx_prev]     # Get labels coordiantes of common detected keypoints from previous frame
                    coor_common_label_curr = detected_labels_src_pts[common_label_idx_curr]          # Get labels coordiantes of common detected keypoints from current frame
                    coor_error = mean_squared_error(coor_common_label_prev, coor_common_label_curr)  # Calculate error between previous and current common keypoints coordinates
                    update_homography = coor_error > keypoints_displacement_mean_tol                 # Check if error surpassed the predefined tolerance level
                else:
                    update_homography = True                                                         
            else:
                update_homography = True

            if  update_homography:
                h, mask = cv2.findHomography(detected_labels_src_pts,                   # Calculate homography matrix
                                              detected_labels_dst_pts)                  
            
            detected_labels_prev = detected_labels.copy()                               # Save current detected keypoint labels for next frame
            detected_labels_src_pts_prev = detected_labels_src_pts.copy()               # Save current detected keypoint to measure error in next frame

            # get boxes of players and ball to map to the field map
            bboxes_p_c_0 = bboxes_p_c[[i==0 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected players (label 0)
            bboxes_p_c_2 = bboxes_p_c[[i==2 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected ball(s) (label 2)

            
            # Get coordinates of detected players on frame (x_center, y_center+h/2)
            detected_ppos_src_pts = bboxes_p_c_0[:,:2]  + np.array([[0]*bboxes_p_c_0.shape[0], bboxes_p_c_0[:,3]/2]).transpose()
            # Get coordinates of the first detected ball (x_center, y_center)
            detected_ball_src_pos = bboxes_p_c_2[0,:2] if bboxes_p_c_2.shape[0]>0 else None

            # Transform players coordinates from frame plane to tactical map plane using the calculated Homography matrix
            pred_dst_pts = []                                                           # Initialize players tactical map coordiantes list
            for pt in detected_ppos_src_pts: # Loop over players frame coordiantes
                # track players how mosh areff
                pt = np.append(np.array(pt), np.array([1]), axis=0)                     # Covert to homogeneous coordiantes
                dest_point = np.matmul(h, np.transpose(pt))                              # Apply homography transofrmation
                dest_point = dest_point/dest_point[2]                                   # Revert to 2D-coordiantes
                pred_dst_pts.append(list(np.transpose(dest_point)[:2]))                 # Update players tactical map coordiantes list
            pred_dst_pts = np.array(pred_dst_pts)

            # Transform ball coordinates from frame plane to tactical map plane using the calculated Homography matrix
            if detected_ball_src_pos is not None:
                pt = np.append(np.array(detected_ball_src_pos), np.array([1]), axis=0)
                dest_point = np.matmul(h, np.transpose(pt))
                dest_point = dest_point/dest_point[2]
                detected_ball_dst_pos = np.transpose(dest_point)

                # Update track ball position history to draw the ball 
                if len(ball_track_history['src'])>0 :
                    if np.linalg.norm(detected_ball_src_pos-ball_track_history['src'][-1])<ball_track_dist_thresh:
                        ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                        ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
                    else:
                        ball_track_history['src']=[(int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1]))]
                        ball_track_history['dst']=[(int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1]))]
                else:
                    ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                    ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
            # Remove oldest tracked ball postion if track exceedes threshold        
            if len(ball_track_history) > max_track_length:
                    ball_track_history['src'].pop(0)
                    ball_track_history['dst'].pop(0)
            # track ball is closest to which player and assign it to a player when it changes players trigger  

        ######### Part 2 ########## 
        # Players Team Prediction #
        ###########################
         
        if (frame_nbr <= 4):
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)                                      # Convert frame to RGB
            obj_palette_list = []                                                                   # Initialize players color palette list
            palette_interval = (0,5)                                                                # Color interval to extract from dominant colors palette (1rd to 5th color)
            annotated_frame = frame                                                                 # Create annotated frame 
    
            for player in player_detections:
                bbox = player.boxes.xyxy.cpu().numpy()[0]
                obj_img = frame_rgb[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
                obj_img_w, obj_img_h = obj_img.shape[1], obj_img.shape[0]
                center_filter_x1 = np.max([(obj_img_w//2)-(obj_img_w//6), 1])
                center_filter_x2 = (obj_img_w//2)+(obj_img_w//6)
                center_filter_y1 = np.max([(obj_img_h//3)-(obj_img_h//6), 1])
                center_filter_y2 = (obj_img_h//3)+(obj_img_h//6)
                center_filter = obj_img[center_filter_y1:center_filter_y2, 
                                            center_filter_x1:center_filter_x2]
                obj_pil_img = Image.fromarray(np.uint8(center_filter))               # Convert to pillow image
                reduced = obj_pil_img.convert("P", palette=Image.Palette.WEB)                   # Convert to web palette (216 colors)
                palette = reduced.getpalette()                                                  # Get palette as [r,g,b,r,g,b,...]
                palette = [palette[3*n:3*n+3] for n in range(256)]                              # Group 3 by 3 = [[r,g,b],[r,g,b],...]
                color_count = [(n, palette[m]) for n,m in reduced.getcolors()]                  # Create list of palette colors with their frequency
                RGB_df = pd.DataFrame(color_count, columns = ['cnt', 'RGB']).sort_values(       # Create dataframe based on defined palette interval
                                          by = 'cnt', ascending = False).iloc[
                                              palette_interval[0]:palette_interval[1],:]
                palette = list(RGB_df.RGB)                                                      # Convert palette to list (for faster processing)
                    
                # Update detected players color palette list
                obj_palette_list.append(palette)

            # lalalalalalala el colors 3awza tetzabat law sama7t neshoof taree2a tanya
            
            ## Calculate distances between each color from every detected player color palette and the predefined teams colors
            players_distance_features = []
            # Loop over detected players extracted color palettes
            for palette in obj_palette_list:
                palette_distance = []
                palette_lab = [skimage.color.rgb2lab([i/255 for i in color]) for color in palette]  # Convert colors to L*a*b space
                # Loop over colors in palette
                for color in palette_lab:
                    distance_list = []
                    # Loop over predefined list of teams colors
                    for c in color_list_lab:
                        distance = skimage.color.deltaE_cie76(color, c)                             # Calculate Euclidean distance in Lab color space
                        distance_list.append(distance)                                              # Update distance list for current color
                    
                    palette_distance.append(distance_list)                                          # Update distance list for current palette
                players_distance_features.append(palette_distance)                                  # Update distance features list
            
            
            ## Predict detected players teams based on distance features
            players_teams_list = []
            # Loop over players distance features
            for i,distance_feats in enumerate(players_distance_features):
                vote_list=[]
                # Loop over distances for each color 
                for dist_list in distance_feats:
                    team_idx = dist_list.index(min(dist_list))                    # Assign team index for current color based on min distance
                    vote_list.append(team_idx)                                                      # Update vote voting list with current color team prediction
                team_idx = max(vote_list, key=vote_list.count)
                team[player_detections[i].tracker_id] = team_idx
                player_detections[i].team = list(colors_dic.keys())[team_idx//2] # "chelsea" or "man city"
                if team_idx % 2 == 0:
                    player_detections[i].position = "Player"
                else :
                    player_detections[i].position = "Goal Keeper"
                players_teams_list.append(team_idx)                      # Predict current player team by vote counting
    
        else :
            annotated_frame = frame
            players_teams_list = []
            for player in player_detections:
                team_idx = team[player.tracker_id]
                player.team = list(colors_dic.keys())[team_idx] # "chelsea" or "man city"
                if team_idx % 2 == 0:
                    player.position = "Player"
                else :
                    player.position = "Goal Keeper"
                players_teams_list.append(team_idx)
            
        #################### Part 3 #####################
        # Updated Frame & Tactical Map With Annotations #
        #################################################
        for player in player_detections:
            bbox = player.boxes.xyxy.numpy()[0]
            annotated_frame = cv2.rectangle(annotated_frame, (int(bbox[0]), int(bbox[1])),  
                                        (int(bbox[2]), int(bbox[3])), (0,0,0), 1)
            if player.team == list(colors_dic.keys())[0]:
                color = (0,0,0)
            else :
                color = (255,255,255)
            cv2.putText(annotated_frame,f" {player.tracker_id}",(int(bbox[0]), int(bbox[1])-10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        ball_color_bgr = (0,0,255)                                                                          # Color (GBR) for ball annotation on tactical map
        j=0                                                                                                 # Initializing counter of detected players
        palette_box_size = 10                                                                               # Set color box size in pixels (for display)
        

        # Loop over all detected object by players detection model
        for i in range(bboxes_p.shape[0]):
            conf = confs_p[i]                                                                               # Get confidence of current detected object
            if labels_p[i]==0:                                                                              # Display annotation for detected players (label 0)

                
                color_rgb = colors_dic["Chelsea"][0]                                                        # Get detected player team color
                color_bgr = color_rgb[::-1]
                # Add tactical map player postion color coded annotation if more than 3 field keypoints are detected
                if len(detected_labels_src_pts)>3:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(pred_dst_pts[j][0]),int(pred_dst_pts[j][1])),
                                          radius=5, color=color_bgr, thickness=-1)

                j+=1                        # Update players counter
            else:                                                                                           # Display annotation for otehr detections (label 1, 2)
                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,0]), int(bboxes_p[i,1])), 
                                               (int(bboxes_p[i,2]), int(bboxes_p[i,3])), (255,255,255), 1)
                cv2.putText(annotated_frame, labels_dic[labels_p[i]] + f" {conf:.2f}",(int(bboxes_p[i,0]), int(bboxes_p[i,1])-10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

                # Add tactical map ball postion annotation if detected
                if detected_ball_src_pos is not None:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(detected_ball_dst_pos[0]), 
                                                   int(detected_ball_dst_pos[1])), radius=5, 
                                                   color=ball_color_bgr, thickness=3)
        # momkin nebadel de be player_detections we neshow el id
        # shoof mawdoo3 el player id el mosh mawgood da
        for i in range(bboxes_k.shape[0]):
            annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_k[i,0]), int(bboxes_k[i,1])),  
                                        (int(bboxes_k[i,2]), int(bboxes_k[i,3])), (0,0,0), 1)
        
        # Plot the ball tracks on tactical map
        if len(ball_track_history['src'])>0:
            points = np.hstack(ball_track_history['dst']).astype(np.int32).reshape((-1, 1, 2))
            tac_map_copy = cv2.polylines(tac_map_copy, [points], isClosed=False, color=(0, 0, 100), thickness=2)
        
        if len(ball_detections) == 1:
            prev_id,prev_idx = player_with_ball_id,player_with_ball_idx
            prev_team = team_with_ball
            player_with_ball_id,player_with_ball_idx = get_player_in_possession(player_detections,ball_detections)
            team_with_ball = player_detections[player_with_ball_idx].team
    
        # dont forget to assign the prevs after you finish
        if prev_id != player_with_ball_id and (frame_nbr - last_frame > 20):
            last_frame = frame_nbr
            prev_team,prev_position = player_detections[prev_idx].team,player_detections[prev_idx].position
            current_team,current_position = player_detections[player_with_ball_idx].team,player_detections[player_with_ball_idx].position
            if prev_id != player_with_ball_id:
                if prev_team != current_team:
                    text += "the ball was intercepted from "+str(prev_team)+" to player "+str(player_with_ball_id)+" of "+str(current_team)
                else :
                    if prev_position != current_position:
                        if prev_position == "Goal Keeper":
                            text += "the ball was passed from the Goal Keeper of "+str(current_team)+" to player "+str(player_with_ball_id)
                        else:
                            text += "the ball was passed from the palyer number "+str(player_with_ball_id)+" to Goal Keeper of "+str(current_team)
                    else :
                        text += "the ball was passed form player "+str(prev_id)+" to player "+str(player_with_ball_id)+" of team "+str(current_team)
            
            else : 
                text += "the ball is still with player number " + str(player_with_ball_id)
            #assign prevs
               
        for player in player_detections:
            if player.tracker_id == 17 :
                print("Box :")
                print(player.boxes.xyxy)
                print("ID :")
                print(player.tracker_id)
                print("Team :")
                print(player.team)
                print("position :")
                print(player.position)
                print("---------------")
        # kol 3 sawani 2ool haga 
        # law el koora 8ayaret id 2ool eh el 7asal

        
        # Combine annotated frame and tactical map in one image with colored border separation
        border_color = [255,255,255]                                                                        # Set border color (BGR)
        annotated_frame=cv2.copyMakeBorder(annotated_frame, 40, 10, 10, 10,                                 # Add borders to annotated frame
                                            cv2.BORDER_CONSTANT, value=border_color)
        tac_map_copy = cv2.copyMakeBorder(tac_map_copy, 70, 50, 10, 10, cv2.BORDER_CONSTANT,                # Add borders to tactical map 
                                           value=border_color)      
        tac_map_copy = cv2.resize(tac_map_copy, (tac_map_copy.shape[1], annotated_frame.shape[0]))          # Resize tactical map
        final_img = cv2.hconcat((annotated_frame, tac_map_copy))                                            # Concatenate both images
        ## Add info annotation
        cv2.putText(final_img, "Tactical Map", (1370,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        cv2.putText(final_img, "Press 'p' to pause & 'q' to quit", (820,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)

        new_frame_time = time.time()                                                                        # Get time after finished processing current frame
        fps = 1/(new_frame_time-prev_frame_time)                                                            # Calculate FPS as 1/(frame proceesing duration)
        prev_frame_time = new_frame_time                                                                    # Save current time to be used in next frame
        cv2.putText(final_img, "FPS: " + str(int(fps)), (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        
        # Display the final annotated frame
        cv2.imshow("YOLOv8 Players and Field Keypoints Detection with Team Prediction and Tactical Map",    
                    final_img)

        # Treat keyboard user inputs ("p" for pause/unpause & "q" for quit)
        key = cv2.waitKey(1)
        # Break the loop if 'q' is pressed
        if key == ord("q"):
            break
        if key == ord('p'):
            cv2.waitKey(-1) #wait until any key is pressed
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined

In [18]:
list(colors_dic.keys())

['Chelsea', 'Man City']

In [22]:
# el denya ma7sooba sa7 bas ana 3awez azboot mawdoo3 el colors da we a7sebo be taree2a mo5talefa
# we 3awez a8ayar el angle beta3 el camera
# we 

{191: 1,
 192: 0,
 193: 0,
 194: 0,
 195: 3,
 196: 0,
 197: 0,
 198: 0,
 199: 0,
 200: 0,
 201: 0,
 202: 0,
 203: 0,
 204: 2,
 205: 2,
 206: 0,
 207: 3,
 208: 0,
 209: 0}

In [19]:
# Open video file
cap = cv2.VideoCapture(video_path)

# Initialize frame counter
frame_nbr = 0

# Set keypoints average displacement tolerance level (in pixels) [set to -1 to always update homography matrix]
keypoints_displacement_mean_tol = 10

# Set confidence thresholds for players and field keypoints detections
player_model_conf_thresh = 0.60
keypoints_model_conf_thresh = 0.70

# Set variable to record the time when we processed last frame 
prev_frame_time = 0
# Set variable to record the time at which we processed current frame 
new_frame_time = 0

# Store the ball track history
ball_track_history = {'src':[],
                      'dst':[]
}

# Count consecutive frames with no ball detected
nbr_frames_no_ball = 0
# Threshold for number of frames with no ball to reset ball track (frames)
nbr_frames_no_ball_thresh = 30
# Distance threshold for ball tracking (pixels)
ball_track_dist_thresh = 100
# Maximum ball track length (detections)
max_track_length = 35

# Loop through the video frames
while cap.isOpened():

    # Update frame counter
    frame_nbr += 1

    # Read a frame from the video
    success, frame = cap.read()

    # Reset tactical map image for each new frame
    tac_map_copy = tac_map.copy()

    # Reset ball tracks
    if nbr_frames_no_ball>nbr_frames_no_ball_thresh:
            ball_track_history['dst'] = []
            ball_track_history['src'] = []

    # Process the frame if it was successfuly read
    if success:
        
        #################### Part 1 ####################
        # Object Detection & Coordiante Transofrmation #
        ################################################

        # Run YOLOv8 players inference on the frame
        results_players = model_players(frame, conf=player_model_conf_thresh)
        # Run YOLOv8 field keypoints inference on the frame
        results_keypoints = model_keypoints(frame, conf=keypoints_model_conf_thresh)

        ## Extract detections information
        bboxes_p = results_players[0].boxes.xyxy.cpu().numpy()                          # Detected players, referees and ball (x,y,x,y) bounding boxes
        bboxes_p_c = results_players[0].boxes.xywh.cpu().numpy()                        # Detected players, referees and ball (x,y,w,h) bounding boxes    
        labels_p = list(results_players[0].boxes.cls.cpu().numpy())                     # Detected players, referees and ball labels list
        confs_p = list(results_players[0].boxes.conf.cpu().numpy())                     # Detected players, referees and ball confidence level
        
        bboxes_k = results_keypoints[0].boxes.xyxy.cpu().numpy()                        # Detected field keypoints (x,y,w,h) bounding boxes
        bboxes_k_c = results_keypoints[0].boxes.xywh.cpu().numpy()                        # Detected field keypoints (x,y,w,h) bounding boxes
        labels_k = list(results_keypoints[0].boxes.cls.cpu().numpy())                   # Detected field keypoints labels list

        # Convert detected numerical labels to alphabetical labels
        detected_labels = [classes_names_dic[i] for i in labels_k]

        # Extract detected field keypoints coordiantes on the current frame
        detected_labels_src_pts = np.array([list(np.round(bboxes_k_c[i][:2]).astype(int)) for i in range(bboxes_k_c.shape[0])])

        # Get the detected field keypoints coordinates on the tactical map
        detected_labels_dst_pts = np.array([keypoints_map_pos[i] for i in detected_labels])


        ## Calculate Homography transformation matrix when more than 4 keypoints are detected
        if len(detected_labels) > 3:
            # Always calculate homography matrix on the first frame
            if frame_nbr > 1:
                # Determine common detected field keypoints between previous and current frames
                common_labels = set(detected_labels_prev) & set(detected_labels)
                # When at least 4 common keypoints are detected, determine if they are displaced on average beyond a certain tolerance level
                if len(common_labels) > 3:
                    common_label_idx_prev = [detected_labels_prev.index(i) for i in common_labels]   # Get labels indexes of common detected keypoints from previous frame
                    common_label_idx_curr = [detected_labels.index(i) for i in common_labels]        # Get labels indexes of common detected keypoints from current frame
                    coor_common_label_prev = detected_labels_src_pts_prev[common_label_idx_prev]     # Get labels coordiantes of common detected keypoints from previous frame
                    coor_common_label_curr = detected_labels_src_pts[common_label_idx_curr]          # Get labels coordiantes of common detected keypoints from current frame
                    coor_error = mean_squared_error(coor_common_label_prev, coor_common_label_curr)  # Calculate error between previous and current common keypoints coordinates
                    update_homography = coor_error > keypoints_displacement_mean_tol                 # Check if error surpassed the predefined tolerance level
                else:
                    update_homography = True                                                         
            else:
                update_homography = True

            if  update_homography:
                h, mask = cv2.findHomography(detected_labels_src_pts,                   # Calculate homography matrix
                                              detected_labels_dst_pts)                  
            
            detected_labels_prev = detected_labels.copy()                               # Save current detected keypoint labels for next frame
            detected_labels_src_pts_prev = detected_labels_src_pts.copy()               # Save current detected keypoint coordiantes for next frame

            bboxes_p_c_0 = bboxes_p_c[[i==0 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected players (label 0)
            bboxes_p_c_2 = bboxes_p_c[[i==2 for i in labels_p],:]                       # Get bounding boxes information (x,y,w,h) of detected ball(s) (label 2)

            # Get coordinates of detected players on frame (x_cencter, y_center+h/2)
            detected_ppos_src_pts = bboxes_p_c_0[:,:2]  + np.array([[0]*bboxes_p_c_0.shape[0], bboxes_p_c_0[:,3]/2]).transpose()
            # Get coordinates of the first detected ball (x_center, y_center)
            detected_ball_src_pos = bboxes_p_c_2[0,:2] if bboxes_p_c_2.shape[0]>0 else None

            # Transform players coordinates from frame plane to tactical map plance using the calculated Homography matrix
            pred_dst_pts = []                                                           # Initialize players tactical map coordiantes list
            for pt in detected_ppos_src_pts:                                            # Loop over players frame coordiantes
                pt = np.append(np.array(pt), np.array([1]), axis=0)                     # Covert to homogeneous coordiantes
                dest_point = np.matmul(h, np.transpose(pt))                              # Apply homography transofrmation
                dest_point = dest_point/dest_point[2]                                   # Revert to 2D-coordiantes
                pred_dst_pts.append(list(np.transpose(dest_point)[:2]))                 # Update players tactical map coordiantes list
            pred_dst_pts = np.array(pred_dst_pts)

            # Transform ball coordinates from frame plane to tactical map plane using the calculated Homography matrix
            if detected_ball_src_pos is not None:
                pt = np.append(np.array(detected_ball_src_pos), np.array([1]), axis=0)
                dest_point = np.matmul(h, np.transpose(pt))
                dest_point = dest_point/dest_point[2]
                detected_ball_dst_pos = np.transpose(dest_point)

                # Update track ball position history
                if len(ball_track_history['src'])>0 :
                    if np.linalg.norm(detected_ball_src_pos-ball_track_history['src'][-1])<ball_track_dist_thresh:
                        ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                        ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
                    else:
                        ball_track_history['src']=[(int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1]))]
                        ball_track_history['dst']=[(int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1]))]
                else:
                    ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                    ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
            # Remove oldest tracked ball postion if track exceedes threshold        
            if len(ball_track_history) > max_track_length:
                    ball_track_history['src'].pop(0)
                    ball_track_history['dst'].pop(0)

        ######### Part 2 ########## 
        # Players Team Prediction #
        ###########################

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)                                      # Convert frame to RGB
        obj_palette_list = []                                                                   # Initialize players color palette list
        palette_interval = (0,5)                                                                # Color interval to extract from dominant colors palette (1rd to 5th color)
        annotated_frame = frame                                                                 # Create annotated frame 

        ## Loop over detected players (label 0) and extract dominant colors palette based on defined interval
        for i, j in enumerate(list(results_players[0].boxes.cls.cpu().numpy())):
            if int(j) == 0:
                bbox = results_players[0].boxes.xyxy.cpu().numpy()[i,:]                         # Get bbox info (x,y,x,y)
                obj_img = frame_rgb[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]       # Crop bbox out of the frame
                obj_img_w, obj_img_h = obj_img.shape[1], obj_img.shape[0]
                center_filter_x1 = np.max([(obj_img_w//2)-(obj_img_w//5), 1])
                center_filter_x2 = (obj_img_w//2)+(obj_img_w//5)
                center_filter_y1 = np.max([(obj_img_h//3)-(obj_img_h//5), 1])
                center_filter_y2 = (obj_img_h//3)+(obj_img_h//5)
                center_filter = obj_img[center_filter_y1:center_filter_y2, 
                                        center_filter_x1:center_filter_x2]
                obj_pil_img = Image.fromarray(np.uint8(center_filter))                          # Convert to pillow image
                    
                reduced = obj_pil_img.convert("P", palette=Image.Palette.WEB)                   # Convert to web palette (216 colors)
                palette = reduced.getpalette()                                                  # Get palette as [r,g,b,r,g,b,...]
                palette = [palette[3*n:3*n+3] for n in range(256)]                              # Group 3 by 3 = [[r,g,b],[r,g,b],...]
                color_count = [(n, palette[m]) for n,m in reduced.getcolors()]                  # Create list of palette colors with their frequency
                RGB_df = pd.DataFrame(color_count, columns = ['cnt', 'RGB']).sort_values(       # Create dataframe based on defined palette interval
                                      by = 'cnt', ascending = False).iloc[
                                          palette_interval[0]:palette_interval[1],:]
                palette = list(RGB_df.RGB)                                                      # Convert palette to list (for faster processing)
                annotated_frame = cv2.rectangle(annotated_frame,                                # Add center filter bbox annotations
                                                (int(bbox[0])+center_filter_x1, 
                                                 int(bbox[1])+ center_filter_y1),  
                                                (int(bbox[0])+center_filter_x2, 
                                                 int(bbox[1])+center_filter_y2), (0,0,0), 2)
                
                # Update detected players color palette list
                obj_palette_list.append(palette)
        
        ## Calculate distances between each color from every detected player color palette and the predefined teams colors
        players_distance_features = []
        # Loop over detected players extracted color palettes
        for palette in obj_palette_list:
            palette_distance = []
            palette_lab = [skimage.color.rgb2lab([i/255 for i in color]) for color in palette]  # Convert colors to L*a*b* space
            # Loop over colors in palette
            for color in palette_lab:
                distance_list = []
                # Loop over predefined list of teams colors
                for c in color_list_lab:
                    distance = skimage.color.deltaE_cie76(color, c)                             # Calculate Euclidean distance in Lab color space
                    distance_list.append(distance)                                              # Update distance list for current color
                palette_distance.append(distance_list)                                          # Update distance list for current palette
            players_distance_features.append(palette_distance)                                  # Update distance features list

        ## Predict detected players teams based on distance features
        players_teams_list = []
        # Loop over players distance features
        for distance_feats in players_distance_features:
            vote_list=[]
            # Loop over distances for each color 
            for dist_list in distance_feats:
                team_idx = dist_list.index(min(dist_list))//nbr_team_colors                     # Assign team index for current color based on min distance
                vote_list.append(team_idx)                                                      # Update vote voting list with current color team prediction
            players_teams_list.append(max(vote_list, key=vote_list.count))                      # Predict current player team by vote counting


        #################### Part 3 #####################
        # Updated Frame & Tactical Map With Annotations #
        #################################################

        ball_color_bgr = (0,0,255)                                                                          # Color (GBR) for ball annotation on tactical map
        j=0                                                                                                 # Initializing counter of detected players
        palette_box_size = 10                                                                               # Set color box size in pixels (for display)
        

        # Loop over all detected object by players detection model
        for i in range(bboxes_p.shape[0]):
            conf = confs_p[i]                                                                               # Get confidence of current detected object
            if labels_p[i]==0:                                                                              # Display annotation for detected players (label 0)
                
                # Display extracted color palette for each detected player
                palette = obj_palette_list[j]                                                               # Get color palette of the detected player
                for k, c in enumerate(palette):
                    c_bgr = c[::-1]                                                                         # Convert color to BGR
                    annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,2])+3,                 # Add color palette annotation on frame
                                                            int(bboxes_p[i,1])+k*palette_box_size),
                                                            (int(bboxes_p[i,2])+palette_box_size,
                                                            int(bboxes_p[i,1])+(palette_box_size)*(k+1)),
                                                              c_bgr, -1)

                team_name = list(colors_dic.keys())[players_teams_list[j]]                                  # Get detected player team prediction
                color_rgb = colors_dic[team_name][0]                                                        # Get detected player team color
                color_bgr = color_rgb[::-1]                                                                 # Convert color to bgr

                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,0]), int(bboxes_p[i,1])),  # Add bbox annotations with team colors
                                                (int(bboxes_p[i,2]), int(bboxes_p[i,3])), color_bgr, 1)
                
                cv2.putText(annotated_frame, team_name + f" {conf:.2f}",                                    # Add team name annotations
                             (int(bboxes_p[i,0]), int(bboxes_p[i,1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                               color_bgr, 2)
                
                # Add tactical map player postion color coded annotation if more than 3 field keypoints are detected
                if len(detected_labels_src_pts)>3:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(pred_dst_pts[j][0]),int(pred_dst_pts[j][1])),
                                          radius=5, color=color_bgr, thickness=-1)

                j+=1                                                                                        # Update players counter
            else:                                                                                           # Display annotation for otehr detections (label 1, 2)
                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,0]), int(bboxes_p[i,1])),  # Add white colored bbox annotations
                                                 (int(bboxes_p[i,2]), int(bboxes_p[i,3])), (255,255,255), 1)
                cv2.putText(annotated_frame, labels_dic[labels_p[i]] + f" {conf:.2f}",                      # Add white colored label text annotations
                            (int(bboxes_p[i,0]), int(bboxes_p[i,1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                              (255,255,255), 2)

                # Add tactical map ball postion annotation if detected
                if detected_ball_src_pos is not None:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(detected_ball_dst_pos[0]), 
                                                   int(detected_ball_dst_pos[1])), radius=5, 
                                                   color=ball_color_bgr, thickness=3)
        for i in range(bboxes_k.shape[0]):
            annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_k[i,0]), int(bboxes_k[i,1])),  # Add bbox annotations with team colors
                                        (int(bboxes_k[i,2]), int(bboxes_k[i,3])), (0,0,0), 1)
        
        # Plot the ball tracks on tactical map
        if len(ball_track_history['src'])>0:
            points = np.hstack(ball_track_history['dst']).astype(np.int32).reshape((-1, 1, 2))
            tac_map_copy = cv2.polylines(tac_map_copy, [points], isClosed=False, color=(0, 0, 100), thickness=2)
        
        # Combine annotated frame and tactical map in one image with colored border separation
        border_color = [255,255,255]                                                                        # Set border color (BGR)
        annotated_frame=cv2.copyMakeBorder(annotated_frame, 40, 10, 10, 10,                                 # Add borders to annotated frame
                                            cv2.BORDER_CONSTANT, value=border_color)
        tac_map_copy = cv2.copyMakeBorder(tac_map_copy, 70, 50, 10, 10, cv2.BORDER_CONSTANT,                # Add borders to tactical map 
                                           value=border_color)      
        tac_map_copy = cv2.resize(tac_map_copy, (tac_map_copy.shape[1], annotated_frame.shape[0]))          # Resize tactical map
        final_img = cv2.hconcat((annotated_frame, tac_map_copy))                                            # Concatenate both images
        ## Add info annotation
        cv2.putText(final_img, "Tactical Map", (1370,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        cv2.putText(final_img, "Press 'p' to pause & 'q' to quit", (820,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)

        new_frame_time = time.time()                                                                        # Get time after finished processing current frame
        fps = 1/(new_frame_time-prev_frame_time)                                                            # Calculate FPS as 1/(frame proceesing duration)
        prev_frame_time = new_frame_time                                                                    # Save current time to be used in next frame
        cv2.putText(final_img, "FPS: " + str(int(fps)), (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
        
        # Display the final annotated frame
        cv2.imshow("YOLOv8 Players and Field Keypoints Detection with Team Prediction and Tactical Map",    
                    final_img)

        # Treat keyboard user inputs ("p" for pause/unpause & "q" for quit)
        key = cv2.waitKey(1)
        # Break the loop if 'q' is pressed
        if key == ord("q"):
            break
        if key == ord('p'):
            cv2.waitKey(-1) #wait until any key is pressed
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'player', 1: 'referee', 2: 'ball'}
orig_img: array([[[ 86,  77,  81],
        [ 83,  75,  79],
        [ 81,  73,  76],
        ...,
        [ 68,  47,  56],
        [ 70,  49,  59],
        [ 72,  51,  60]],

       [[ 88,  80,  83],
        [ 87,  79,  82],
        [ 83,  75,  79],
        ...,
        [ 79,  58,  67],
        [ 79,  58,  67],
        [ 79,  58,  67]],

       [[ 94,  86,  89],
        [ 93,  84,  88],
        [ 89,  81,  84],
        ...,
        [122, 101, 110],
        [108,  87,  96],
        [102,  81,  90]],

       ...,

       [[ 77, 123, 103],
        [ 60, 105,  86],
        [ 51, 103,  82],
        ...,
        [ 42, 117,  91],
        [ 42, 117,  91],
        [ 42, 117,  91]],

       [[ 47, 100,  79],
        [ 52, 104,  83],
        [ 52, 108,  86],
        ...,
        [ 42, 117,  91],
        [ 42, 117,  91]

AttributeError: 'Results' object has no attribute 'tracker_id'. See valid attributes below.

    A class for storing and manipulating inference results.

    Args:
        orig_img (numpy.ndarray): The original image as a numpy array.
        path (str): The path to the image file.
        names (dict): A dictionary of class names.
        boxes (torch.tensor, optional): A 2D tensor of bounding box coordinates for each detection.
        masks (torch.tensor, optional): A 3D tensor of detection masks, where each mask is a binary image.
        probs (torch.tensor, optional): A 1D tensor of probabilities of each class for classification task.
        keypoints (List[List[float]], optional): A list of detected keypoints for each object.

    Attributes:
        orig_img (numpy.ndarray): The original image as a numpy array.
        orig_shape (tuple): The original image shape in (height, width) format.
        boxes (Boxes, optional): A Boxes object containing the detection bounding boxes.

In [67]:
players_teams_list = []
for player in player_detections:
    team_idx = team[player.tracker_id]
    player.team = list(colors_dic.keys())[team_idx//2] # "chelsea" or "man city"
    if team_idx % 2 == 0:
        player.position = "Player"
    else :
        player.position = "Goal Keeper"
    players_teams_list.append(team_idx)

In [ ]:
# how about you ask about the action and let the agent answer you
# whether it is the score , time , ball passed
# and have a mode that is activated to commentate
# 

In [ ]:
                # Display extracted color palette for each detected player
                # palette = obj_palette_list[j]                                                               # Get color palette of the detected player
                # for k, c in enumerate(palette):
                #     c_bgr = c[::-1]                                                                         # Convert color to BGR

                # team_name = list(colors_dic.keys())[players_teams_list[j]]                                  # Get detected player team prediction
                # color_rgb = colors_dic[team_name][0]                                                        # Get detected player team color
                # color_bgr = color_rgb[::-1]                                                                 # Convert color to bgr

                # annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i,0]), int(bboxes_p[i,1])),  # Add bbox annotations with team colors
                #                                 (int(bboxes_p[i,2]), int(bboxes_p[i,3])), color_bgr, 1)
                
                # cv2.putText(annotated_frame, team_name + f" {conf:.2f}",                                    # Add team name annotations
                #              (int(bboxes_p[i,0]), int(bboxes_p[i,1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                #                color_bgr, 2)